In [28]:
import os
import pickle
import pandas as pd

In [7]:
path = os.getcwd()

In [8]:
with open(path + '/data/raw.pickle', 'rb') as f:
    data = pickle.load(f)

In [10]:
df = data[data['star'] != str(0.0)]

In [11]:
idxs = []
for title in df['title']:
    idx = '친환경' in title or '유기농' in title or '무농약' in title
    idxs.append(idx)
    
eco_df = df[idxs].reset_index(drop=True)

idxs = [not x for x in idxs]

ord_df = df[idxs].reset_index(drop=True)

In [12]:
#  평점 비교
import numpy as np

In [16]:
round(np.mean([float(x) for x in eco_df['star']]),2) # 친환경 관련 작물 평점 평균

8.44

In [15]:
round(np.mean([float(x) for x in ord_df['star']]),2) # 일반 작물 평점 평균

7.07

In [17]:
# 리뷰 감성분석
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px

In [18]:
def corpusToSentimentScore(corpus,sentiment,model):
    corpus_embeddings = model.encode([corpus])
    corpus_embeddings_2 = model.encode([sentiment])
    return cosine_similarity(corpus_embeddings, corpus_embeddings_2)[0][0]

In [19]:
def corpusToSentimentScoreList(corpus,SentimentList,model):
    sentimentScoreList = []
    for sentiment in sentimentList:
        sentimentScore = corpusToSentimentScore(corpus,sentiment,model)
        sentimentScoreList.append(sentimentScore)
    return sentimentScoreList

In [20]:
def corpusListToMeanScoreList(corpusList,sentimentList,model):
    agg = np.zeros(len(sentimentList))
    for corpus in corpusList:
        agg += np.array(corpusToSentimentScoreList(corpus,sentimentList,model))
    return agg / len(corpusList)

In [21]:
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
sentimentList = ['맛있다','크다','비싸다','건강하다','빠르다']

In [22]:
eco_df_reviews = [x for x in eco_df['review'].explode()]
ord_df_reviews = [x for x in ord_df['review'].explode()]

In [23]:
MeanScoreList_eco = corpusListToMeanScoreList(eco_df_reviews,sentimentList,model) 

In [24]:
MeanScoreList_ord = corpusListToMeanScoreList(ord_df_reviews,sentimentList,model) 

In [25]:
corpus = eco_df_reviews[0]
sentimentScoreList = corpusToSentimentScoreList(corpus,sentimentList,model)

In [34]:
df = pd.DataFrame(dict(r= sentimentScoreList ,theta=sentimentList))

fig = px.line_polar(df, r='r', theta='theta', line_close=True)

fig.update_traces(fill='toself')

fig.show()
print(corpus)

비닐에 싸여서 왔고 안에 충격완화를 위한 완충재가 따로 있었어요. 근데 고구마 상태가 전에 주문했을땐 전부 괜찮았는데 이번엔 완전 물러서 집을때 귤 집는것 같이 물컹한 고구마가 두개 정도 있었어요. 맛은 좋습니다. 크기도 한두개 먹으면 배부를만큼 크고 실했는데 좀 아쉽네요.


In [30]:
ord_df = pd.DataFrame({'score':MeanScoreList_ord,'sentiment':sentimentList})
ord_df['type'] = 'ord' 

In [31]:
eco_df = pd.DataFrame({'score':MeanScoreList_eco,'sentiment':sentimentList})
eco_df['type'] = 'eco' 

In [32]:
df2 = pd.concat([ord_df,eco_df])

In [33]:
#그래프 그리기
fig = px.line_polar(df2, r="score", theta="sentiment", color="type", line_close=True)

# 내부 색칠하기
fig.update_traces(fill='toself')

fig.show()